In [1]:
import yolort
print(yolort.__version__)
print(yolort.__file__)

0.7.0a0
/home/guest1/LISHUFEI/jupyter_code/yolov5-rt-stack/yolort/__init__.py


In [2]:
import argparse
import platform
import sys
import time
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
FILE = Path().resolve()
ROOT = FILE.parents[0]  # YOLOrt root directory
print(str(ROOT))
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
print(str(ROOT)in sys.path)

/home/guest1/LISHUFEI/jupyter_code/yolov5-rt-stack
True


### 配置环境

In [5]:
import os
import cv2

import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # PCI_BUS_ID” # 按照PCI_BUS_ID顺序从0开始排列GPU设备 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  #设置当前使用的GPU设备仅为0号设备  设备名称为'/gpu:0'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 导入模型和定义的函数

In [6]:
from yolort.models.yolo import YOLO
from yolort.utils import Visualizer, get_image_from_url, read_image_to_tensor, check_dataset
from yolort.v5 import load_yolov5_model, letterbox, non_max_suppression, scale_coords, attempt_download
from yolort.v5.utils.downloads import safe_download
from yolort.v5.utils.dataloaders import *
from yolort.v5.utils.general import colorstr

### 加载数据

In [7]:
data = ROOT / 'data/coco128.yaml'  # dataset.yaml path
data = check_dataset(data)

In [8]:
# print(data)

In [9]:
img_source = "https://huggingface.co/spaces/zhiqwang/assets/resolve/main/bus.jpg"
# img_source = "https://huggingface.co/spaces/zhiqwang/assets/resolve/main/zidane.jpg"
img_raw = get_image_from_url(img_source)

In [10]:
print(img_raw)

[[[122 148 172]
  [120 146 170]
  [125 153 177]
  ...
  [157 170 184]
  [158 171 185]
  [158 171 185]]

 [[127 153 177]
  [124 150 174]
  [127 155 179]
  ...
  [158 171 185]
  [159 172 186]
  [159 172 186]]

 [[128 154 178]
  [126 152 176]
  [126 154 178]
  ...
  [158 171 185]
  [158 171 185]
  [158 171 185]]

 ...

 [[185 185 191]
  [182 182 188]
  [179 179 185]
  ...
  [114 107 112]
  [115 105 111]
  [116 106 112]]

 [[157 157 163]
  [180 180 186]
  [185 186 190]
  ...
  [107  97 103]
  [102  92  98]
  [108  98 104]]

 [[112 112 118]
  [160 160 166]
  [169 170 174]
  ...
  [ 99  89  95]
  [ 96  86  92]
  [102  92  98]]]


In [11]:
# image = letterbox(img_raw, new_shape=(img_size, img_size), stride=stride)[0]
# image = read_image_to_tensor(image)
# image = image.to(device)
# image = image[None]
# print(image.size())

In [12]:
img_size = 640
stride = 64
score_thresh = 0.25
batch_size = 32
nms_thresh = 0.45
single_cls=False # treat as single-class dataset
rect = False
workers=6  # max dataloader workers (per RANK in DDP mode)

In [13]:
# print(batch_size)

In [14]:
#images_source = 
#我现在需要做的是不是把images/train2017的图片加载进来？
task = 'val'
task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
loader,dataset = create_dataloader(data[task],
                               img_size,
                               stride,
                               batch_size,
                               single_cls,
                               rect=rect,
                               workers=workers,
                               prefix=colorstr(f'{task}: '))


val: Scanning '/home/guest1/LISHUFEI/jupyter_code/datasets/coco128/labels/train2017.cache' images and labels... 128 found, 0 missing, 2 empty, 


In [15]:
print(len(dataset))

128


In [16]:
# create_dataloader的返回值是什么？
print(len(loader))  # batch_size num = 4
ime = dataset[0] # 读取到的图像是一个元组类型的  ime[2]
print(dataset[0])
print(ime[2])
print(type(ime[2]))

4
(tensor([[[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]],

        [[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]],

        [[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]]], dtype=torch.uint8), tensor([[ 0.00000, 45.00000,  0.47949,  0.64158,  0.95561,  0.44662],
        [ 0.0

### 加载模型权重

In [17]:
# 如果项目路径下没有的话去指定的路径下下载
model_path = 'yolov5s.pt'
checkpoint_path = attempt_download(model_path)

### 加载模型

In [18]:
model_yolort = YOLO.load_from_yolov5(
    checkpoint_path,
    score_thresh=score_thresh,
    nms_thresh=nms_thresh,
    version="r6.0",
)
model_yolort = model_yolort.eval()
model_yolort = model_yolort.to(device)

In [19]:
for i, im in enumerate(dataset):
    im_path = im[2]
    img_raw = cv2.imread(im_path)  # opencv set the BGR order as the default
    assert img_raw is not None, f"Not Found Image: {im_path}"
    im = letterbox(img_raw, new_shape=(img_size, img_size), stride=stride)[0]
    im = read_image_to_tensor(im)
    im = im.to(device)
    im = im[None]
    print(im.size())
    with torch.no_grad():
        yolort_dets = model_yolort(im)

torch.Size([1, 3, 512, 640])


/home/guest1/anaconda3/envs/pytorch-lsf/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 640, 448])
torch.Size([1, 3, 640, 576])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 640, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 640, 448])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 576, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 576, 640])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 448, 640])
torch.Size([1, 3, 640, 512])
torch.Size([1, 3, 640, 448])
torch.Size([1, 3, 512, 640])
torch.Size([1, 3, 576, 640])
torch.Size([1,

In [22]:
print(data["path"])

../datasets/coco128
